 Decision Trees and Forests are LOVE

- Logistic Regression
- KNN
- Naive Bayes
- SVM
- Decision Tree
- RandomForestClassifier
- AdaBoost
- Gradient Boosting
- XGBoost
- Feature Importance

In [ ]:
# Load the functions
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score

import matplotlib.pyplot as plt
import seaborn as sns


1. Pirate Attacks
You are provided with a dataset on vessels that have been attacked by pirates (pirate_data.csv). Your task is to predict whether an attack is likely to be successful.

Compare the performance of KNN, Naive Bayes, SVM and Random Forest.





1.1. Clean and preprocess the data. This includes:
Select features. Caution: Some features might lead to “data leakage” …

One-hot encode categorical variables.

How do you handle the date?

Aggregate categories with few occurrences.

In [ ]:
# DATA UPLOAD
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/MyDrive/04_MachineLearning/pirate_data.csv')
df.head(3)

Mounted at /content/drive


,Unnamed: 0,DATETIME (LOCAL),DATETIME (UTC),DATE (LT),DATE (UTC),REGION,COUNTRY,INCIDENT TYPE,VESSEL TYPE,MAERSK?,VESSEL ACTIVITY LOCATION,LAT,LONG,TIMEZONE,TIME OF DAY,ATTACKS,ATTACK SUCCESS
0,1,2016-01-30 23:34:00,2016-01-30 22:34:00,2016-01-30,2016-01-30,WEST AFRICA,NIGERIA,"FAILED ATTACK, HIJACKING",CONTAINER SHIP,0,OFF SHORE,3.000000,6.250000,Africa/Lagos,EVENING,1,0
1,2,2016-01-29 23:40:00,2016-01-29 22:40:00,2016-01-29,2016-01-29,WEST AFRICA,NIGERIA,KIDNAP/RANSOM AT SEA,PRODUCT TANKER,0,OFF SHORE,2.500000,5.750000,Africa/Lagos,EVENING,1,1
2,3,2016-01-28 03:00:00,2016-01-28 02:00:00,2016-01-28,2016-01-28,SOUTH ASIA,INDIA,"FAILED ATTACK, ROBBERY",PRODUCT TANKER,0,ANCHORAGE,22.816667,70.116667,Asia/Kolkata,NIGHT,1,0


In [ ]:
# DATA CLEANING
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                802 non-null    int64  
 1   DATETIME (LOCAL)          582 non-null    object 
 2   DATETIME (UTC)            582 non-null    object 
 3   DATE (LT)                 802 non-null    object 
 4   DATE (UTC)                802 non-null    object 
 5   REGION                    802 non-null    object 
 6   COUNTRY                   802 non-null    object 
 7   INCIDENT TYPE             802 non-null    object 
 8   VESSEL TYPE               638 non-null    object 
 9   MAERSK?                   802 non-null    int64  
 10  VESSEL ACTIVITY LOCATION  676 non-null    object 
 11  LAT                       802 non-null    float64
 12  LONG                      802 non-null    float64
 13  TIMEZONE                  802 non-null    object 
 14  TIME OF DA

In [ ]:
df.isnull().sum()

Unnamed: 0                    0
DATETIME (LOCAL)            220
DATETIME (UTC)              220
DATE (LT)                     0
DATE (UTC)                    0
REGION                        0
COUNTRY                       0
INCIDENT TYPE                 0
VESSEL TYPE                 164
MAERSK?                       0
VESSEL ACTIVITY LOCATION    126
LAT                           0
LONG                          0
TIMEZONE                      0
TIME OF DAY                   0
ATTACKS                       0
ATTACK SUCCESS                0
dtype: int64

In [ ]:
df.columns

Index(['Unnamed: 0', 'DATETIME (LOCAL)', 'DATETIME (UTC)', 'DATE (LT)',
       'DATE (UTC)', 'REGION', 'COUNTRY', 'INCIDENT TYPE', 'VESSEL TYPE',
       'MAERSK?', 'VESSEL ACTIVITY LOCATION', 'LAT', 'LONG', 'TIMEZONE',
       'TIME OF DAY', 'ATTACKS', 'ATTACK SUCCESS'],
      dtype='object')

In [ ]:
df['MAERSK?'].value_counts()

0    794
1      8
Name: MAERSK?, dtype: int64

In [ ]:
df['ATTACKS'].value_counts()

1    802
Name: ATTACKS, dtype: int64

In [ ]:
df['REGION'].value_counts()

SOUTH EAST ASIA                 313
WEST AFRICA                     288
SOUTH ASIA                       67
HORN OF AFRICA/ GULF OF ADEN     34
MIDDLE EAST                      25
SOUTH AMERICA                    23
NORTH AFRICA                     18
NORTH EAST ASIA                  10
EAST AFRICA                       8
CARIBBEAN                         7
EUROPE                            5
PACIFIC                           3
NORTH AMERICA                     1
Name: REGION, dtype: int64

In [ ]:
# Aggregate countries 
other_countries=list(df['REGION'].value_counts().index[df['REGION'].value_counts()<10]) 
other_countries

['EAST AFRICA', 'CARIBBEAN', 'EUROPE', 'PACIFIC', 'NORTH AMERICA']

In [ ]:

def change_to_other(region):
  if region in other_countries:
    return "OTHER"
  else:
    return region

df["REGION"] = df["REGION"].apply(change_to_other)

In [ ]:
df["REGION"].value_counts()

SOUTH EAST ASIA                 313
WEST AFRICA                     288
SOUTH ASIA                       67
HORN OF AFRICA/ GULF OF ADEN     34
MIDDLE EAST                      25
OTHER                            24
SOUTH AMERICA                    23
NORTH AFRICA                     18
NORTH EAST ASIA                  10
Name: REGION, dtype: int64

In [ ]:
# df['VESSEL TYPE'].unique()
df['VESSEL TYPE'].value_counts()

# There are many columns with small value counts 

BULK CARRIER                          134
PRODUCT TANKER                        107
CONTAINER SHIP                         60
TUG                                    34
TANKER                                 34
FISHING VESSEL                         33
MERCHANT VESSEL                        30
GENERAL CARGO VESSEL                   28
CRUDE OIL TANKER                       28
PASSENGER VESSEL                       25
SUPPLY VESSEL                          25
CHEMICAL TANKER                        15
UNKNOWN                                10
LPG TANKER                             10
VEHICLE CARRIER                         7
TRAWLER                                 7
BARGE                                   5
FISHING TRAWLER                         4
DREDGER                                 4
HEAVY LIFT VESSEL                       3
REEFER                                  3
FISHING DHOW                            2
FLOATING STORAGE OFFLOADING VESSEL      2
WOOD CHIPS CARRIER                

In [ ]:
# Aggregate the small occurence values as "others"
other_vessels = list(df["VESSEL TYPE"].value_counts().index[df["VESSEL TYPE"].value_counts() < 10])

def change_to_other(vessel_type):
    if vessel_type in other_vessels:
        return "OTHER"
    else:
        return vessel_type

df["VESSEL TYPE"] = df["VESSEL TYPE"].apply(change_to_other)
df["VESSEL TYPE"].value_counts()

BULK CARRIER            134
PRODUCT TANKER          107
OTHER                    65
CONTAINER SHIP           60
TANKER                   34
TUG                      34
FISHING VESSEL           33
MERCHANT VESSEL          30
GENERAL CARGO VESSEL     28
CRUDE OIL TANKER         28
PASSENGER VESSEL         25
SUPPLY VESSEL            25
CHEMICAL TANKER          15
LPG TANKER               10
UNKNOWN                  10
Name: VESSEL TYPE, dtype: int64

In [ ]:
# Create column weekday based on DATE column : Monday is 0 and Sunday is 6.
df['WEEKDAY'] = pd.DatetimeIndex(df["DATE (UTC)"]).weekday.astype(str)

In [ ]:
# Recode numeric values to categorical names. 
nums = ["0","1","2","3","4","5","6"]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df['WEEKDAY'] = df['WEEKDAY'].replace(nums, days)
df.head()

,Unnamed: 0,DATETIME (LOCAL),DATETIME (UTC),DATE (LT),DATE (UTC),REGION,COUNTRY,INCIDENT TYPE,VESSEL TYPE,MAERSK?,VESSEL ACTIVITY LOCATION,LAT,LONG,TIMEZONE,TIME OF DAY,ATTACKS,ATTACK SUCCESS,WEEKDAY
0,1,2016-01-30 23:34:00,2016-01-30 22:34:00,2016-01-30,2016-01-30,WEST AFRICA,NIGERIA,"FAILED ATTACK, HIJACKING",CONTAINER SHIP,0,OFF SHORE,3.000000,6.250000,Africa/Lagos,EVENING,1,0,Saturday
1,2,2016-01-29 23:40:00,2016-01-29 22:40:00,2016-01-29,2016-01-29,WEST AFRICA,NIGERIA,KIDNAP/RANSOM AT SEA,PRODUCT TANKER,0,OFF SHORE,2.500000,5.750000,Africa/Lagos,EVENING,1,1,Friday
2,3,2016-01-28 03:00:00,2016-01-28 02:00:00,2016-01-28,2016-01-28,SOUTH ASIA,INDIA,"FAILED ATTACK, ROBBERY",PRODUCT TANKER,0,ANCHORAGE,22.816667,70.116667,Asia/Kolkata,NIGHT,1,0,Thursday
3,4,NaN,NaN,2016-01-27,2016-01-27,HORN OF AFRICA/ GULF OF ADEN,SOMALIA,SUSPICIOUS ACTIVITY AT SEA,NaN,0,OFF SHORE,13.998747,54.112792,Asia/Riyadh,UNKNOWN,1,0,Wednesday
4,5,2016-01-25 23:55:00,2016-01-25 22:55:00,2016-01-25,2016-01-25,SOUTH AMERICA,VENEZUELA,THEFT AT ANCHORAGE,GENERAL CARGO VESSEL,0,ANCHORAGE,10.267500,-63.432500,America/Caracas,EVENING,1,1,Monday


In [ ]:
df['VESSEL ACTIVITY LOCATION'].value_counts()

OFF SHORE    421
ANCHORAGE    198
BERTH         57
Name: VESSEL ACTIVITY LOCATION, dtype: int64

In [ ]:
# Column drop columns criteria: less categorical characteristics / data leakeage: "INCIDENT TYPE" /'MAERSK?': too much imbalances in value counts. 
# Extract Date,

X=df.drop(columns=['ATTACK SUCCESS','Unnamed: 0','COUNTRY', 'DATETIME (LOCAL)', 'DATETIME (UTC)', 'DATE (LT)','DATE (UTC)','LAT', 'LONG', 
                   'TIMEZONE','MAERSK?','ATTACKS','INCIDENT TYPE'])
X.columns
X.head()

,REGION,VESSEL TYPE,VESSEL ACTIVITY LOCATION,TIME OF DAY,WEEKDAY
0,WEST AFRICA,CONTAINER SHIP,OFF SHORE,EVENING,Saturday
1,WEST AFRICA,PRODUCT TANKER,OFF SHORE,EVENING,Friday
2,SOUTH ASIA,PRODUCT TANKER,ANCHORAGE,NIGHT,Thursday
3,HORN OF AFRICA/ GULF OF ADEN,NaN,OFF SHORE,UNKNOWN,Wednesday
4,SOUTH AMERICA,GENERAL CARGO VESSEL,ANCHORAGE,EVENING,Monday


In [ ]:
# Check imbalances on class in dependent variable
y=df['ATTACK SUCCESS']
y.value_counts()

1    500
0    302
Name: ATTACK SUCCESS, dtype: int64

In [ ]:
# Splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y) #I forgot what stratify does.... 
X_train.shape, X_test.shape

((561, 5), (241, 5))

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 561 entries, 43 to 352
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   REGION                    561 non-null    object
 1   VESSEL TYPE               437 non-null    object
 2   VESSEL ACTIVITY LOCATION  467 non-null    object
 3   TIME OF DAY               561 non-null    object
 4   WEEKDAY                   561 non-null    object
dtypes: object(5)
memory usage: 26.3+ KB


In [ ]:
# Working on categorical features 

categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

categorical_features

['REGION', 'VESSEL TYPE', 'VESSEL ACTIVITY LOCATION', 'TIME OF DAY', 'WEEKDAY']

In [ ]:
# categorical feature encoder
categorical_transformer = Pipeline(steps=[
                                          ("cat_imputer", SimpleImputer(strategy='constant',
                                                                        fill_value='Not Available')),
                                          ("onehot", OneHotEncoder(handle_unknown="ignore"))
                                          ])
categorical_transformer


Pipeline(steps=[('cat_imputer',
                 SimpleImputer(fill_value='Not Available',
                               strategy='constant')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
# Pipeline with categorical transformers 
preprocessor = ColumnTransformer(transformers=[
                                               ("cat", categorical_transformer, 
                                                       categorical_features)
                                               ])
preprocessor

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(fill_value='Not '
                                                                           'Available',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['REGION', 'VESSEL TYPE',
                                  'VESSEL ACTIVITY LOCATION', 'TIME OF DAY',
                                  'WEEKDAY'])])


1.2. Run all four models on the data

KNN

In [ ]:
# Import and define KNN function
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn

KNeighborsClassifier()

In [ ]:
pipeline_knn = Pipeline([("pre_process", preprocessor), 
                         ("model", knn)])
pipeline_knn

Pipeline(steps=[('pre_process',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'Available',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['REGION', 'VESSEL TYPE',
                                                   'VESSEL ACTIVITY LOCATION',
                                                   'TIME OF DAY',
                                                   'WEEKDAY'])])),
                ('model', KNeighborsClassifier())])

In [ ]:
pipeline_knn.fit(X_train, y_train)
y_pred = pipeline_knn.predict(X_test)

class_labels = pipeline_knn.named_steps['model'].classes_

print(classification_report(y_test, y_pred))

pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

              precision    recall  f1-score   support

           0       0.45      0.44      0.45        91
           1       0.67      0.68      0.67       150

    accuracy                           0.59       241
   macro avg       0.56      0.56      0.56       241
weighted avg       0.59      0.59      0.59       241



,0,1
0,40,51
1,48,102


In [ ]:
scores = {}

In [ ]:
scores['knn'] = round(f1_score(y_test, y_pred, average='weighted'), 3)
print(scores['knn'])

0.588


Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB()
nb

BernoulliNB()

In [ ]:
pipeline_nb = Pipeline([("pre_process", preprocessor), 
                         ("model", nb)])
pipeline_nb

Pipeline(steps=[('pre_process',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'Available',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['REGION', 'VESSEL TYPE',
                                                   'VESSEL ACTIVITY LOCATION',
                                                   'TIME OF DAY',
                                                   'WEEKDAY'])])),
                ('model', BernoulliNB())])

In [ ]:
pipeline_nb.fit(X_train, y_train)
y_pred = pipeline_nb.predict(X_test)

class_labels = pipeline_nb.named_steps['model'].classes_

print(classification_report(y_test, y_pred))

pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

              precision    recall  f1-score   support

           0       0.51      0.54      0.52        91
           1       0.71      0.69      0.70       150

    accuracy                           0.63       241
   macro avg       0.61      0.61      0.61       241
weighted avg       0.63      0.63      0.63       241



,0,1
0,49,42
1,47,103


In [ ]:
scores['nb'] = round(f1_score(y_test, y_pred, average='weighted'), 3)
print(scores['nb'])

0.633


SVM ML Pipeline

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=42, max_iter=10000)
svc

LinearSVC(max_iter=10000, random_state=42)

In [ ]:
pipeline_svc = Pipeline([("pre_process", preprocessor), 
                         ("model", svc)])
pipeline_svc

Pipeline(steps=[('pre_process',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'Available',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['REGION', 'VESSEL TYPE',
                                                   'VESSEL ACTIVITY LOCATION',
                                                   'TIME OF DAY',
                                                   'WEEKDAY'])])),
                ('model', LinearSVC(max_iter=10000, random_state=42))])

In [ ]:
pipeline_svc.fit(X_train, y_train)
y_pred = pipeline_svc.predict(X_test)

class_labels = pipeline_svc.named_steps['model'].classes_

print(classification_report(y_test, y_pred))

pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

              precision    recall  f1-score   support

           0       0.58      0.38      0.46        91
           1       0.69      0.83      0.76       150

    accuracy                           0.66       241
   macro avg       0.64      0.61      0.61       241
weighted avg       0.65      0.66      0.65       241



,0,1
0,35,56
1,25,125


In [ ]:
scores['svm'] = round(f1_score(y_test, y_pred, average='weighted'), 3)
scores

{'knn': 0.588, 'nb': 0.633, 'svm': 0.645}

Random Forest

In [ ]:
# Trees dont like scaled, they are free-spirited creatures ! So, take out the scale function from my pipe line. 

# categorical_transformer = Pipeline(steps=[
#                                           ("cat_imputer", SimpleImputer(strategy='constant',
#                                                                         fill_value='Not Available')),
#                                           ("onehot", OneHotEncoder(handle_unknown="ignore"))
#                                           ])

# preprocessor_new = ColumnTransformer(transformers=[

#                                                ("cat", categorical_transformer, 
#                                                        categorical_features)
#                                                ])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf

RandomForestClassifier(random_state=42)

In [ ]:
pipeline_rf = Pipeline([("pre_process", preprocessor), 
                         ("model", rf)])
pipeline_rf

Pipeline(steps=[('pre_process',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'Available',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['REGION', 'VESSEL TYPE',
                                                   'VESSEL ACTIVITY LOCATION',
                                                   'TIME OF DAY',
                                                   'WEEKDAY'])])),
                ('model', RandomForestClassifier(random_state=42))])

In [ ]:
pipeline_rf.fit(X_train, y_train)
y_pred = pipeline_rf.predict(X_test)

class_labels = pipeline_rf.named_steps['model'].classes_

print(classification_report(y_test, y_pred))

pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

              precision    recall  f1-score   support

           0       0.49      0.42      0.45        91
           1       0.68      0.74      0.71       150

    accuracy                           0.62       241
   macro avg       0.59      0.58      0.58       241
weighted avg       0.61      0.62      0.61       241



,0,1
0,38,53
1,39,111


In [ ]:
scores['rf'] = round(f1_score(y_test, y_pred, average='weighted'), 3)
scores

{'knn': 0.588, 'nb': 0.633, 'svm': 0.645, 'rf': 0.611}

1.3. Tune the models and identify the best performing one

In [ ]:
X_train.columns

Index(['REGION', 'VESSEL TYPE', 'VESSEL ACTIVITY LOCATION', 'TIME OF DAY',
       'WEEKDAY'],
      dtype='object')

In [ ]:
# option 1
from sklearn.ensemble import RandomForestClassifier
importances = rf.feature_importances_
feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest_importances = pd.Series(importances, index=X_train.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

ValueError: ignored

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
X_train.columns
y_test
y

0      0
1      1
2      0
3      0
4      1
      ..
797    0
798    1
799    1
800    1
801    1
Name: ATTACK SUCCESS, Length: 802, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=3,
                            max_features='auto', 
                            min_samples_leaf=4,
                            # bootstrap=True, 
                            n_jobs=-1, 
                            random_state=0)
rf.fit(X, y)

ValueError: ignored